In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from deeplab.oaisys_utils import OAISYS_LABELS
import semsegcluster.data.scannet
from IPython.display import display

from semsegcluster.oaisys_eval_munkres import get_measurements, get_ood
from deeplab.oaisys_ood_detection import get_measurements as get_measurements_ood
from semsegcluster.settings import EXP_OUT
from semsegcluster.data.nyu_depth_v2 import TRAINING_LABEL_NAMES, NYU40_COLORS
from semsegcluster.sacred_utils import get_incense_loader
from semsegcluster.segmentation_metrics import SegmentationMetric
from deeplab.oaisys_utils import OAISYS_LABELS
import detectron2 as det2
import detectron2.utils.visualizer
# loader = get_incense_loader()

In [ ]:
subset='oaisys_trajectory_grass'
methods=['label','pred395-merged-ensemble_pred-seg122','merged-ensemble_pred-seg122', 'ensemble_pred', 'seg122', 'pred400-merged-ensemble_pred-seg122', 'pred403-merged-ensemble_pred-seg122', 'dino365']
methods=['label','merged-ensemble_pred-seg122', 'ensemble_pred', 'pred','seg122', 'pred400-merged-ensemble_pred-seg122', 'pred403-merged-ensemble_pred-seg122', 'dino365']

OAISYS_LABELS[15] = 'Grass-Field'
m = get_measurements(subset=subset, pretrained_id='deeplab', methods=methods)
df = pd.DataFrame.from_dict({name: {
    OAISYS_LABELS[l]: m[name]['assigned_iou'][l]
    for l in range(37)
    if not np.isnan(m['label']['assigned_iou'][l])
} for name in m if not name == 'label'}).T
df = df.fillna(0)
df['mean'] = df.mean(axis=1)
df['v_score'] = [m[name]['v_score'] for name in df.index]
pd.set_option("display.float_format", lambda f: f"{f:0.2f}")
#print(df.to_latex(float_format=lambda x: f'{x * 100:.0f}'))
df.sort_values(by='mean', ascending=False)[:45]
# print(df.sort_values(by='mean', ascending=False)[:45].to_latex(float_format=lambda x: f'{x * 100:.0f}'))

In [ ]:
subset='oaisys_trajectory3'
methods=['label','pred396-merged-ensemble_pred-seg107','merged-ensemble_pred-seg107', 'ensemble_pred', 'seg107', 'pred401-merged-ensemble_pred-seg107', 'pred402-merged-ensemble_pred-seg107', 'dino368']
methods=['label','merged-ensemble_pred-seg107', 'ensemble_pred', 'pred', 'seg107', 'pred401-merged-ensemble_pred-seg107', 'pred402-merged-ensemble_pred-seg107', 'dino368']
OAISYS_LABELS[15] = 'Sand'
m = get_measurements(subset=subset, pretrained_id='deeplab', methods=methods)
df = pd.DataFrame.from_dict({name: {
    OAISYS_LABELS[l]: m[name]['assigned_iou'][l]
    for l in range(37)
    if not np.isnan(m['label']['assigned_iou'][l])
} for name in m if not name == 'label'}).T
df = df.fillna(0)
df['mean'] = df.mean(axis=1)
df['v_score'] = [m[name]['v_score'] for name in df.index]
pd.set_option("display.float_format", lambda f: f"{f:0.2f}")
#print(df.to_latex(float_format=lambda x: f'{x * 100:.0f}'))
df.sort_values(by='mean', ascending=False)[:45]
# print(df.sort_values(by='mean', ascending=False)[:45].to_latex(float_format=lambda x: f'{x * 100:.0f}'))

In [ ]:
subset='rugd'
methods=['ensemble_label', 'ensemble_pred', 'pred402-rugd']
OAISYS_LABELS[15] = 'Sand'
m = get_measurements(subset=subset, pretrained_id='deeplab', methods=methods, only_newclass=True)
df = pd.DataFrame.from_dict({name: {
    OAISYS_LABELS[l]: m[name]['assigned_iou'][l]
    for l in range(37)
    if not np.isnan(m['ensemble_label']['assigned_iou'][l])
} for name in m if not name == 'ensemble_label'}).T
df = df.fillna(0)
df['mean'] = df.mean(axis=1)
df['v_score'] = [m[name]['v_score'] for name in df.index]
pd.set_option("display.float_format", lambda f: f"{f:0.2f}")
#print(df.to_latex(float_format=lambda x: f'{x * 100:.0f}'))
df.sort_values(by='mean', ascending=False)[:45]

In [ ]:
subset='rugd_sand'
methods=['label', 'ensemble_pred', 'seg449', 'merged-ensemble_pred-seg449_-4.39', 'dino454', 'merged-ensemble_pred-dino454_-4.39', 'pred456-merged-ensemble_pred-dino454_-4.39']
OAISYS_LABELS[15] = 'Sand'
m = get_measurements(subset=subset, pretrained_id='deeplab', methods=methods, only_newclass=False)
df = pd.DataFrame.from_dict({name: {
    OAISYS_LABELS[l]: m[name]['assigned_iou'][l]
    for l in range(37)
    if not np.isnan(m['label']['assigned_iou'][l])
} for name in m if not name == 'label'}).T
df = df.fillna(0)
df['mean'] = df.mean(axis=1)
df['v_score'] = [m[name]['v_score'] for name in df.index]
pd.set_option("display.float_format", lambda f: f"{f:0.2f}")
#print(df.to_latex(float_format=lambda x: f'{x * 100:.0f}'))
df.sort_values(by='mean', ascending=False)[:45]
# print(df.sort_values(by='mean', ascending=False)[:45].to_latex(float_format=lambda x: f'{x * 100:.0f}'))


In [ ]:
subset='oaisys_trajectory3'
methods=['ensemble_maxlogit-pp']
uncert_treshold = -5.5
get_ood(subset=subset, pretrained_id='deeplab', methods=methods, uncert_treshold=uncert_treshold)
# print('done')
# print(measurements)
# with pd.option_context('display.float_format', '{:0.4f}'.format):
#     print(pd.DataFrame.from_dict(measurements).T, flush=True)

In [ ]:
subsets = ['rugd_sand', 'oaisys_trajectory_grass', 'oaisys_trajectory3']
methods=['ensemble_maxlogit-pp', 'maxlogit-pp', 'norm_maxlogit-pp', 'gmm_max_log_prob_100_11']

eval_values = ['AP', 'AUROC', 'ideal_threshold', 'Precision', 'Recall', 'Accuracy', 'F1']

subsets = ['rugd_sand']
eval_values = ['AP', 'AUROC', 'F1', 'ideal_threshold']



# cen_input_measurements = {
#     'oaisys_trajectory_grass': {
#         'AUROC': 0.7814402430642234,
#         'AP': 0.31355944696811,
#     },
#     'oaisys_trajectory3': {
#         'AUROC': 0.4996484766932421,
#         'AP': 0.38695004408845196,
#     },
# }

# plot the results
n_sub = 10000
all_measurements = {}
measurements = get_measurements_ood(subsets=subsets, pretrained_id='deeplab', methods=methods, only_newclass=True)
for subset in subsets:
    for method in methods:
        try:
            threshold = measurements[subset][method]['Threshold']
        except:
            continue
        fig, ax = plt.subplots(1, 2, figsize=(12, 4))
        sens = measurements[subset][method]['TPR']
        spec = 1 - measurements[subset][method]['FPR']
        g_mean = np.sqrt(sens * spec)

        ideal_index = np.argmax(g_mean)
        ideal_threshold = threshold[ideal_index]
        measurements[subset][method]['ideal_threshold'] = ideal_threshold
        fp = measurements[subset][method]['FPR'][ideal_index]
        measurements[subset][method]['FP'] = fp
        tp = measurements[subset][method]['TPR'][ideal_index]
        measurements[subset][method]['TP'] = tp
        measurements[subset][method]['prescision'] = tp / (tp + fp)

        fpr_sub = measurements[subset][method]['FPR'][0::n_sub]
        tpr_sub = measurements[subset][method]['TPR'][0::n_sub]
        sens_sub = sens[0::n_sub]
        spec_sub = spec[0::n_sub]
        g_mean_sub = g_mean[0::n_sub]
        threshold_sub = threshold[0::n_sub]

        ax[0].set_title(f'ROC Curve')
        ax[0].set_xlabel('FPR')
        ax[0].set_ylabel('TPR')
        ax[0].plot(fpr_sub, tpr_sub, label='ROC')
        ax[0].legend()
        ax[1].set_title(f'Threshold evaluation')
        ax[1].set_xlabel('Treshold')
        ax[1].plot(threshold_sub, sens_sub, label='Sensitivity')
        ax[1].plot(threshold_sub, spec_sub, label='Specificity')
        ax[1].plot(threshold_sub, g_mean_sub, label='G-Mean')
        ax[1].legend()
        # add tilte to figure
        fig.suptitle(f'{method}, dataset: {subset}')
# cen_measurement = {
#     'AUROC': cen_input_measurements[subset]['AUROC'],
#     'AP': cen_input_measurements[subset]['AP'],
# }
# cen_measurements = {
#     'CEN_paper': cen_measurement,
# }
# measurements.update(cen_measurements)
# measurements['CEN_paper']['AUROC'] = 0.7
# measurements['CEN_paper']['AUPR'] = 0.2
# with pd.option_context('display.float_format', '{:0.4f}'.format):
#     print(pd.DataFrame.from_dict(measurements).T, flush=True)

for subset in subsets:
    m = measurements[subset]
    df = pd.DataFrame.from_dict({method: {
        eval_value: m[method][eval_value]
        for eval_value in eval_values
    } for method in methods
      if m[method]}).T
    # df = df.fillna(0)
    # df['mean'] = df.mean(axis=1)
    # df['v_score'] = [m[name]['v_score'] for name in df.index]
    pd.set_option("display.float_format", lambda f: f"{f:0.2f}")
    #print(df.to_latex(float_format=lambda x: f'{x * 100:.0f}'))
    print(df.sort_values(by='AUROC',ascending=False).style.set_caption(subset).to_latex())

df.sort_values(by='AUROC',ascending=False)


    
